# Network 더 쉽게 만들기(nn.Sequential)

우선 기본적인 code를 적어보겠습니다.

In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader 
import numpy as np 

import torchvision 
import torchvision.transforms as transforms 
import torchvision.datasets as datasets

import matplotlib 
import matplotlib.pyplot as plt 

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./cifar10_data',
                                                            train=True,
                                                            download=True,
                                                            transform=transform)
 
testset = torchvision.datasets.CIFAR10(root='./cifar10_data',
                                                            train=False,
                                                            download=True,
                                                            transform=transform)

trainloader = DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### class를 정의할 때 nn.Sequential을 이용하는 방법

이전에 사용했던 방법은 아래와 같이 사용하였습니다.

```python
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.conv2 = nn.Conv2d(64, 30, 5)
        self.fc1 = nn.Linear(30*5*5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x), inplace=True)
        x = F.max_pool2d(x, (2,2))
        x = F.relu(self.conv2(x), inplace=True)
        x = F.max_pool2d(x, (2,2))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x), inplace=True)
        x = F.relu(self.fc2(x), inplace=True)
        return x
```

하지만 이렇게 일일히 써야하고, 일일히 쓰게되면 forward에서도 변수들을 일일히 다 써줘야합니다. layer가 길어지면 계속 쓰기가 귀찮기도 하고 하나하나 잡아주기가 벅찹니다. 

따라서 **self.layer1 = nn.Sequential()** 와 같이 nn.Sequential()을 이용하여 아래와 같이 layer를 쉽게 구성할 수 있습니다.

```python
    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 32, 5),
        nn.ReLU(inplace=True),
        nn.Conv2d(32, 64, 3),
        nn.ReLU(inplace=True),
        nn.Conv2d(64, 128, 3),
        nn.Conv2d(128, 128, 3),
        nn.ReLU(inplace=True),
        nn.Conv2d(128, 256, 3),
        nn.MaxPool2d(2)
    )
```

직접 code로 작성해보겠습니다.

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.layer1 = nn.Sequential( 
            nn.Conv2d(3, 32, 5),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, 3),
            nn.Conv2d(128, 128, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3),
            nn.MaxPool2d(2)
        )
        
        self.layer2_1 = nn.Sequential(
            nn.Conv2d(256, 512, 7, 1, 2),
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )
        
        self.layer2_2 = nn.Sequential(
            nn.Conv2d(256, 512, 5, 1, 1),
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )
        
        self.layer2_3 = nn.Sequential(
            nn.Conv2d(256, 512, 3),
            nn.Conv2d(512, 64, 1),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(3*64*4*4, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 10)
        )
        
    def forward(self, x):
        print(x.data.shape)
        x = self.layer1(x)
        x1 = self.layer2_1(x)
        x2 = self.layer2_2(x)
        x3 = self.layer2_3(x)
        
        x = torch.cat((x1, x2, x3), 1)
        
        x = x.view(x.shape[0], -1)        
        x = self.fc(x)
        return x

# GPU
# net = Net().cuda()
net = Net()

### x = torch.cat((x1, x2, x3), 1)

```python

torch.cat(seq, dim=0, out=None) → Tensor
        
x = torch.randn(2, 3)
>>> x

  0.5983 -0.0341  2.4918
  1.5981 -0.5265 -0.8735
 [torch.FloatTensor of size (2,3)]

>>> torch.cat((x, x, x), 0)

  0.5983 -0.0341  2.4918
  1.5981 -0.5265 -0.8735
  0.5983 -0.0341  2.4918
  1.5981 -0.5265 -0.8735
  0.5983 -0.0341  2.4918
  1.5981 -0.5265 -0.8735
 [torch.FloatTensor of size (6,3)]

>>> torch.cat((x, x, x), 1)

  0.5983 -0.0341  2.4918  0.5983 -0.0341  2.4918  0.5983 -0.0341  2.4918
  1.5981 -0.5265 -0.8735  1.5981 -0.5265 -0.8735  1.5981 -0.5265 -0.8735
 [torch.FloatTensor of size (2,9)]


 x1, x2, x3는 B x C x H x W로 되어있습니다. 여기서 1은 'C(channel)로 정렬하겠다.' 라는 뜻입니다.

        x = torch.cat((x1, x2, x3), 1)
```        

### x = x.view(x.shape[0], -1)

```python
여기서도 x는 B x C x H x W로 되어있습니다. x.shape[0]은 B(batch_size)를 뜻하고
batch_size를 제외하고 '나머지는 fc해야하니까 1차원(1줄)으로 쭉 펼쳐주세요.' 라는 뜻입니다.

        x = x.view(x.shape[0], -1)        
```        

이어서 결과값이 잘 나오는지 확인해보겠습니다.

In [4]:
a = torch.rand(1,3,32,32)
print(a.shape)

a = Variable(a)
print(a.shape)

torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])


In [5]:
net = Net()

out = net(a)
print(out.shape)

torch.Size([1, 3, 32, 32])
torch.Size([1, 10])


우리가 정한대로 [1, 10]로 잘 나오는 것을 볼 수 있습니다.

In [6]:
print(out)

Variable containing:
1.00000e-02 *
  2.0835  2.0330  2.9364  0.5986 -1.3144  2.5163 -1.4628 -3.6521  0.5933 -0.5305
[torch.FloatTensor of size 1x10]



***

# Model 저장하고 불러와서 다시 쓰기

위에서 out을 출력한 이유는 'net = Net()'의 값을 저장했다가 다시 불러서 연산을 했을 때 똑같이 나오도록 하기 위해서 출력한 것입니다. 

Network를 쓰고, 저장하고, 불러오는 것을 해보겠습니다.

### 저장

In [7]:
torch.save(net.state_dict(), './save_model/cnn.pth')

### 불러오기

In [8]:
model = Net()
model.load_state_dict(torch.load('./save_model/cnn.pth'))

### 결과 확인하기

In [9]:
out = model(a)
print(out)

torch.Size([1, 3, 32, 32])
Variable containing:
1.00000e-02 *
  2.0835  2.0330  2.9364  0.5986 -1.3144  2.5163 -1.4628 -3.6521  0.5933 -0.5305
[torch.FloatTensor of size 1x10]

